连续变量单调分箱(假设因变量为好坏用户，其中1为坏用户，0为好用户)

In [ ]:
#实现连续型变量单调分箱，在等频的基础上：其中DF是导入的数据，Y是因变量的字段名，X是自变量的字段名
def mono_woe(DF,Y,X):
    r = 0
    bad=Y.sum()      #坏客户数(假设因变量列为1的是坏客户)
    good=Y.count()-bad  #好客户数
    n=5
    while np.abs(r) < 1:
        d1 = pd.DataFrame({"X": X, "Y": Y, "Bucket": pd.qcut(X, n,duplicates='drop')})
        d2 = d1.groupby('Bucket', as_index = False)
        r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
        n = n - 1
    d3 = pd.DataFrame(d2.X.min(), columns = ['min'])
    d3['min']=d2.min().X    
    d3['max'] = d2.max().X
    d3['sum'] = d2.sum().Y
    d3['total'] = d2.count().Y
    d3['bad_rate'] = d2.mean().Y
    d3['group_rate']=d3['total']/(bad+good)
    d3['woe']=np.log((d3['bad_rate']/(1-d3['bad_rate']))/(bad/good))
    d3['iv']=(d3['sum']/bad-((d3['total']-d3['sum'])/good))*d3['woe']
    iv=d3['iv'].sum()
    d3['iv_sum']=iv
    woe=list(d3['woe'].round(6))   #返回woe可以看到每组对应的woe值
    cut=list(d3['min'].round(6))
    cut1=list(d3['max'].round(6))
    cut.append(cut1[-1]+1)   #返回cut可以看到分箱切割点
    x_woe=pd.cut(X,cut,right=False,labels=woe)
    return  d3,cut,woe,iv,x_woe

In [ ]:
#调用函数，也可以在pycharm中调用
mono_woe(df)

无法单调分箱的连续变量

In [ ]:
#如果某个连续型变量含有高频值的时候，会无法进行分割，可通过下面方法进行分箱
def unmono_woe(DF,Y,X):
    bad=Y.sum()      #坏客户数(假设因变量列为1的是坏客户)
    good=Y.count()-bad  #好客户数
    dn1 = pd.DataFrame({"X": X, "Y": Y, "Bucket": pd.cut(X, 50)})
    dn2 = dn1.groupby('Bucket', as_index = False)
    dn3 = pd.DataFrame(dn2.X.min(), columns = ['min'])
    dn3['min']=dn2.min().X    
    dn3['max'] = dn2.max().X
    dn3['sum'] = dn2.sum().Y
    dn3['total'] = dn2.count().Y
    while (1):
            if  (len(dn3)>5):
                dn3_min_index = dn3[dn3.total == min(dn3.total)].index.values[0]
                if (dn3_min_index!=0):    #最小值非第一行的情况
                    dn3.iloc[dn3_min_index-1, 1] = dn3.iloc[dn3_min_index, 1] 
                    dn3.iloc[dn3_min_index-1, 2] = dn3.iloc[dn3_min_index, 2] +dn3.iloc[dn3_min_index-1, 2]
                    dn3.iloc[dn3_min_index-1, 3] = dn3.iloc[dn3_min_index, 3] +dn3.iloc[dn3_min_index-1, 3]
                    dn3=dn3.drop([dn3_min_index])
                    dn3= dn3.reset_index(drop=True)
                else:    #最小值是第一行的情况
                    dn3.iloc[dn3_min_index+1, 0] = dn3.iloc[dn3_min_index, 0] 
                    dn3.iloc[dn3_min_index+1, 2] = dn3.iloc[dn3_min_index, 2] +dn3.iloc[dn3_min_index+1, 2]
                    dn3.iloc[dn3_min_index+1, 3] = dn3.iloc[dn3_min_index, 3] +dn3.iloc[dn3_min_index+1, 3]
                    dn3=dn3.drop([dn3_min_index])
                    dn3= dn3.reset_index(drop=True)
            else:
                break
    dn3['bad_rate'] =dn3['sum']/dn3['total']
    dn3['group_rate']=dn3['total']/(bad+good)
    dn3['woe']=np.log((dn3['bad_rate']/(1-dn3['bad_rate']))/(bad/good))
    dn3['iv']=(dn3['sum']/bad-((dn3['total']-dn3['sum'])/good))*dn3['woe']
    iv=dn3['iv'].sum()
    dn3['iv_sum']=iv
    woe=list(dn3['woe'].round(6)) 
    cut=list(dn3['min'].round(6))
    cut1=list(dn3['max'].round(6))
    cut.append(cut1[-1]+1)
    x_woe=pd.cut(X,cut,right=False,labels=woe) #取apply_reject_sum这个变量，此时返回dn3：
    return dn3

类别型变量分箱

In [ ]:
def cate_woe(DF,Y,X):
    bad=Y.sum()      #坏客户数
    good=Y.count()-bad  #好客户数
    d1 = pd.DataFrame({"X": X, "Y": Y})
    d2 = d1.groupby('X', as_index =True)
    d3 = pd.DataFrame()
    d3['sum'] = d2.sum().Y
    d3['total'] = d2.count().Y
    for c in range(d3.shape[0])[::-1]:
        if ((d3.iloc[c,1]-d3.iloc[c,0])==0) or (d3.iloc[c,0]==0):
            d3.iloc[c-1,0]=d3.iloc[c-1,0]+d3.iloc[c,0]
            d3.iloc[c-1,1]=d3.iloc[c-1,1]+d3.iloc[c,1]
            d3.drop(d3.index[c],inplace=True)
        else:
            continue
        
    d3['min']=d3.index  
    d3['max'] = d3.index
    d3['bad_rate'] =d3['sum']/d3['total']
    d3['group_rate']=d3['total']/(bad+good)
    d3['woe']=np.log((d3['bad_rate']/(1-d3['bad_rate']))/(bad/good))
    d3['iv']=(d3['sum']/bad-((d3['total']-d3['sum'])/good))*d3['woe']
    iv=d3['iv'].sum()
    d3['iv_sum']=iv
    d3=d3[['min','max','sum','total','bad_rate','group_rate','woe','iv','iv_sum']]
    woe=list(d3['woe'].round(6))
    cut=list(d3.index)
    x_woe=X.replace(cut,woe)   #取max_apply_request_amount_level这个类别型变量，返回d3
    return d3

In [ ]:
#在具体项目中可以将上述三种方法结合，详见Vscode